# Toolz 

In [151]:
import cytoolz
import json
import utils

In [152]:
df = utils.read_csv('data/measured_real_power.csv')
stack = df.stack()
stack = stack.reset_index()
stack.columns = ['timestamp', 'attr:meter', 'power']
stack['attr'] = stack['attr:meter'].apply(lambda x: x.split(":")[0])
stack['meter'] = stack['attr:meter'].apply(lambda x: x.split(":")[1])
stack.head()

,timestamp,attr:meter,power,attr,meter
0,2013-07-01 00:00:00 UTC,measured_real_power:triplex_meter_0,510.912,measured_real_power,triplex_meter_0
1,2013-07-01 00:00:00 UTC,measured_real_power:triplex_meter_1,510.912,measured_real_power,triplex_meter_1
2,2013-07-01 00:00:00 UTC,measured_real_power:triplex_meter_2,510.912,measured_real_power,triplex_meter_2
3,2013-07-01 00:00:00 UTC,measured_real_power:triplex_meter_3,547.924,measured_real_power,triplex_meter_3
4,2013-07-01 00:00:00 UTC,measured_real_power:triplex_meter_4,510.912,measured_real_power,triplex_meter_4


In [153]:
data = stack.to_dict(orient="records")

In [154]:
data[0]

{'attr': 'measured_real_power',
 'attr:meter': 'measured_real_power:triplex_meter_0',
 'meter': 'triplex_meter_0',
 'power': 510.912,
 'timestamp': '2013-07-01 00:00:00 UTC'}

## A query with `cytoolz`

```sql
        select meter, sum(power)
        from data
        where timestamp > 2013-07-01 12:00:00
```

We need to filter for sure

In [165]:
from toolz import groupby, valmap, filter, pluck, map

In [166]:
tmp = filter(lambda x: x['timestamp']>"2013-07-03 14:00:00", data)
tmp = list(tmp)
tmp[:3]

[{'attr': 'measured_real_power',
  'attr:meter': 'measured_real_power:triplex_meter_0',
  'meter': 'triplex_meter_0',
  'power': -1591.14,
  'timestamp': '2013-07-03 14:00:00 UTC'},
 {'attr': 'measured_real_power',
  'attr:meter': 'measured_real_power:triplex_meter_1',
  'meter': 'triplex_meter_1',
  'power': 848.546,
  'timestamp': '2013-07-03 14:00:00 UTC'},
 {'attr': 'measured_real_power',
  'attr:meter': 'measured_real_power:triplex_meter_2',
  'meter': 'triplex_meter_2',
  'power': 487.74699999999996,
  'timestamp': '2013-07-03 14:00:00 UTC'}]

In [171]:
tmp = filter(lambda x: x['timestamp']>"2013-07-03 14:00:00", data)
grp = groupby('meter', tmp)
list(grp.items())[:3]

[('triplex_meter_12',
  [{'attr': 'measured_real_power',
    'attr:meter': 'measured_real_power:triplex_meter_12',
    'meter': 'triplex_meter_12',
    'power': 493.20099999999996,
    'timestamp': '2013-07-03 14:00:00 UTC'},
   {'attr': 'measured_real_power',
    'attr:meter': 'measured_real_power:triplex_meter_12',
    'meter': 'triplex_meter_12',
    'power': 493.20099999999996,
    'timestamp': '2013-07-03 14:01:00 UTC'},
   {'attr': 'measured_real_power',
    'attr:meter': 'measured_real_power:triplex_meter_12',
    'meter': 'triplex_meter_12',
    'power': 493.2,
    'timestamp': '2013-07-03 14:02:00 UTC'},
   {'attr': 'measured_real_power',
    'attr:meter': 'measured_real_power:triplex_meter_12',
    'meter': 'triplex_meter_12',
    'power': 493.19199999999995,
    'timestamp': '2013-07-03 14:03:00 UTC'},
   {'attr': 'measured_real_power',
    'attr:meter': 'measured_real_power:triplex_meter_12',
    'meter': 'triplex_meter_12',
    'power': 493.19199999999995,
    'timestamp':

In [172]:
tmp = filter(lambda x: x['timestamp']>"2013-07-03 14:00:00", data)
grp = groupby('meter', tmp)
valmap(lambda x: sum(pluck("power", x)), grp)  

{'triplex_meter_0': 1937749.9135499983,
 'triplex_meter_1': 923009.383420006,
 'triplex_meter_10': 4063816.573249995,
 'triplex_meter_11': 10721733.465999953,
 'triplex_meter_12': 10134763.07999998,
 'triplex_meter_13': 1813934.812860001,
 'triplex_meter_14': 12087116.23899994,
 'triplex_meter_15': 3665306.3793299985,
 'triplex_meter_16': 3286511.8874900104,
 'triplex_meter_17': 1076415.081970002,
 'triplex_meter_18': 11799570.613000074,
 'triplex_meter_19': 1940545.5860200059,
 'triplex_meter_2': 11391604.097299986,
 'triplex_meter_3': 9469707.960100042,
 'triplex_meter_4': 10894509.894199926,
 'triplex_meter_5': 1889763.09076001,
 'triplex_meter_6': 10020544.384399954,
 'triplex_meter_7': 9293305.711199963,
 'triplex_meter_8': 2664985.374593723,
 'triplex_meter_9': 10289587.465000005}